***IMPORTS Y ACCESO A DATOS DE CSV BICIS***

In [1]:
#Importación de paquetes y lectura de csv:
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import random
from sklearn import model_selection
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
import copy

random.seed(27)

#Leemos el archivo csv:
dataBicis = pd.read_csv('hour.csv')
dataBicis = dataBicis.drop("dteday", axis=1)
dataBicis = dataBicis.drop("instant", axis=1)
atributosBicis = dataBicis.loc[:, 'season':'registered']
objetivoBicis = dataBicis['cnt']

#Dividimos los datos en conjuntos de entrenamiento y prueba
(atributos_entrenamientoBicis, atributos_pruebaBicis,
 objetivo_entrenamientoBicis, objetivo_pruebaBicis) = model_selection.train_test_split(
    atributosBicis, objetivoBicis, test_size=.33)

atributos_entrenamientoBicis = atributos_entrenamientoBicis.values
objetivo_entrenamientoBicis = objetivo_entrenamientoBicis.values
atributos_pruebaBicis = atributos_pruebaBicis.values
objetivo_pruebaBicis= objetivo_pruebaBicis.values

rangosBicis = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in dataBicis.columns:
    valor_min = dataBicis[columna].min()
    valor_max = dataBicis[columna].max()
    rangosBicis[columna] = {'min': valor_min, 'max': valor_max}
    
Iterac=10
atributosAcambiar=5

***MÉTODO LIME***

In [2]:
def metodo_lime(N,x,k,model,rangos):
    random.seed(27)
    Xprima=[]
    R=[]
    W=[]
    Yprima= []
    for i in range (0,N):
        atributosAPerturbar=seleccionar_atributos_aleatorios(x,k)
        xprima = perturbar_muestra(x,atributosAPerturbar,rangos)
        w = distancia_entre_muestras(x,xprima)
        r=atributosAPerturbar
        Xprima.append(xprima)
        R.append(r)
        W.append(w)
        yprima = prediccion_de_modelo(xprima,model)
        Yprima.append(yprima)
    W=np.array(W)
    W = W.reshape(W.shape[0])
    G = entrenar_modelo_ridge(R,Yprima,W)
    return G.coef_

#Se seleccionan los k atributos aleatorios a modificar
def seleccionar_atributos_aleatorios(dataframe, k):
    columnas = dataframe.columns.tolist()
    seleccionados = random.sample(range(len(columnas)), k)
    lista = [0] * len(columnas)
    for i in seleccionados:
        lista[i] = 1
    #print(lista)
    return lista

#Se perturba la muestra original de forma que los atributos seleccionados pueden variar en un rango en función del valor máximo y mínimo de dicho atributo
def perturbar_muestra(x, atributosAPerturbar,rangos):
    xprima = x.copy()
    for i in range(0,len(atributosAPerturbar)):
        if atributosAPerturbar[i] == 1:
            rango_min = xprima[i] - rangos[list(rangos.keys())[i]]['min']
            rango_max = rangos[list(rangos.keys())[i]]['max'] - xprima[i]
            xprima[i] = random.uniform(rango_min, rango_max)
    return xprima

# Se calcula la distancia coseno entre la muestra original y la perturbada
def distancia_entre_muestras(x,xprima):
    return cosine_distances(x.values, xprima.values)

#Se predice el resultado de la muestra con el modelo elegido
def prediccion_de_modelo(xprima,model):
    if not isinstance(model, (RandomForestRegressor, RandomForestClassifier)):
        xprima = DMatrix(xprima)
    y_pred = model.predict(xprima)
    return y_pred


#Se entrena el modelo de regresión Ridge con las muestras perturbadas y los pesos calculados
def entrenar_modelo_ridge(R, Yprima, W):
    modelo_entrenado = Ridge(alpha=1.0) 
    modelo_entrenado.fit(R, Yprima, sample_weight=W)
    return modelo_entrenado

***MODELOS BICIS***

In [3]:
#RandomForest
#Creamos el modelo RandomForest (En este caso es Regressor ya que se trata de una tarea de regresión)
modelRandomForestBicis = RandomForestRegressor(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestBicis.fit(atributos_entrenamientoBicis, objetivo_entrenamientoBicis)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainBicis = xgb.DMatrix(atributos_entrenamientoBicis, label=objetivo_entrenamientoBicis)
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
# Definimos los parámetros del modelo XGBoost
paramsBicis = {
    'objective': 'reg:squarederror',  # Objetivo para regresión
    'eval_metric': 'rmse'  # Métrica de evaluación, error cuadrático medio (RMSE)
}

# Entrenamos el modelo XGBoost
modelXGBoostBicis = xgb.train(paramsBicis, dtrainBicis)

In [4]:
#Se calcula la lista de predicciones y de explicaciones del modelo XGBoost con el csv de Bicis:
clon_model_XGboost_bicis = copy.deepcopy(modelXGBoostBicis)
prediccionesBicisXG = clon_model_XGboost_bicis.predict(dtestBicis)
explicacionesBicisXG = []
atributosAcambiar = 5
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_bicis,rangosBicis)
    explicacionesBicisXG.append(explicacion)
print(explicacionesBicisXG)

[array([[-0.02589458,  0.08545998, -0.00350272, -0.03088337, -0.0615492 ,
        -0.02641141, -0.01445935,  0.07573846,  0.0695528 , -0.02689603,
        -0.07330277,  0.07373793,  0.06131485, -0.10290458]]), array([[ 0.32203392, -1.23724239, -2.7287287 , -0.21810058,  1.28908935,
        -2.15192265,  2.49117442,  0.28810704, -1.47256081,  0.17935273,
         0.66591509, -0.49503206, -0.89170185,  3.9596165 ]]), array([[ 1.69603097, -1.31053749, -3.35844063, -0.30463199, -0.1929211 ,
        -3.44701693,  5.07948213,  0.58637745, -1.55160372, -0.19660414,
        -0.57478414, -1.52040559,  0.40509829,  4.68995689]]), array([[ 3.23165922, -3.74109999, -8.94455893, -0.61121786,  1.09070224,
        -7.93273973, 11.84080043,  1.84792077, -5.02871933,  0.20030067,
        -1.03194177, -3.87810432, -0.17480476, 13.13180334]]), array([[  2.68410643, -15.47906962, -53.47815401,  -6.91906923,
         33.39950819, -19.67483724,  32.75219051,   7.94744688,
        -32.79138568,  13.44212377,

In [5]:
#Se calcula la lista de predicciones y de explicaciones del modelo RandomForest con el csv de Bicis:
prediccionesBicisRF = modelRandomForestBicis.predict(atributos_pruebaBicis)
explicacionesBicisRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestBicis,rangosBicis)
    explicacionesBicisRF.append(explicacion)
print(explicacionesBicisRF)

[array([[-0.21114225,  0.26610947, -0.42608786, -0.02854883, -0.0632201 ,
        -0.44708142,  0.22453112,  0.07859656,  0.24896868, -0.0247712 ,
        -0.06990527,  0.25935201,  0.05665926,  0.13653983]]), array([[-0.23265089, -0.77723445, -3.26685156, -0.17603363,  1.24648145,
        -2.65725034,  2.49185004,  0.34888966, -1.02051867,  0.21678379,
         0.75252568,  0.10483783, -0.98594419,  3.95511528]]), array([[ 1.66891939, -1.27545903, -3.55049088, -0.32610112, -0.17219482,
        -3.64350117,  5.23376169,  0.60063123, -1.52290127, -0.2095557 ,
        -0.58440719, -1.48542494,  0.41471265,  4.85201116]]), array([[ 4.75867109, -4.92305821, -8.01303618, -0.81675156,  0.98691903,
        -7.01210903, 12.30581959,  2.07532697, -6.27484924,  0.09208779,
        -1.47192979, -5.2994909 ,  0.20757403, 13.38482641]]), array([[  3.65560147, -16.46916686, -55.41375037,  -8.15959265,
         34.15529244, -20.68990518,  33.97306634,   9.66062654,
        -34.2563192 ,  13.28212627,

***VALIDACIÓN DE MODELO***

In [6]:
#Función para evaluar la precisión de un modelo
def evaluate_model(model, Xentrenamiento, yentrenamiento, Xtest, ytest):
    # Validación cruzada
    cv_scores = cross_val_score(model, Xentrenamiento, yentrenamiento, cv=2)
    mean_cv_score = np.mean(cv_scores)
    
    print("Puntuación media de validación cruzada:", mean_cv_score)
    
    #.score
    score = model.score(Xtest, ytest)
    print("Puntuación en el conjunto de prueba:", score)

In [7]:
evaluate_model(modelRandomForestBicis, atributos_entrenamientoBicis, objetivo_entrenamientoBicis, atributos_pruebaBicis, objetivo_pruebaBicis)

Puntuación media de validación cruzada: 0.9994269671883972
Puntuación en el conjunto de prueba: 0.9997683866949413


******

***ACCESO A DATOS DE CSV PEN***

In [8]:
# Lectura del archivo CSV de Pen HandWriting
pen = pd.read_csv('pendigits.csv', header=0, names=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15','a16',"class-objective"])

atributosPen = pen.loc[:, 'a1':'a16']
objetivoPen = pen['class-objective']

rangosPen = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in pen.columns:
    valor_min = pen[columna].min()
    valor_max = pen[columna].max()
    rangosPen[columna] = {'min': valor_min, 'max': valor_max}

In [9]:
Iterac=10
atributosAcambiar=5
(atributos_entrenamientoPen, atributos_pruebaPen,
 objetivo_entrenamientoPen, objetivo_pruebaPen) = model_selection.train_test_split(
    atributosPen, objetivoPen, test_size=.33,stratify=objetivoPen)

atributos_entrenamientoPen = atributos_entrenamientoPen.values
objetivo_entrenamientoPen = objetivo_entrenamientoPen.values
atributos_pruebaPen = atributos_pruebaPen.values
objetivo_pruebaPen= objetivo_pruebaPen.values

print(atributos_entrenamientoPen)

[[ 29 100  20 ...  47   0  44]
 [ 87  90  38 ...  24  27   0]
 [  0  95  43 ...  35 100  38]
 ...
 [ 48  89   8 ... 100 100  88]
 [100 100  53 ...  30  12  32]
 [ 69 100  27 ...  22   0  28]]


***MODELOS PEN***

In [10]:
#Instanciamos el modelo RandomForest
modelRandomForestPen = RandomForestClassifier(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestPen.fit(atributos_entrenamientoPen, objetivo_entrenamientoPen)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainPen = xgb.DMatrix(atributos_entrenamientoPen, label=objetivo_entrenamientoPen)
dtestPen = xgb.DMatrix(atributos_pruebaPen)

# Definimos los parámetros del modelo XGBoost
paramsPen = {
    'objective': 'multi:softmax',  # Función objetivo para clasificación multiclase
    'num_class': 10,       # Número de clases en tu problema
    'eval_metric': 'merror' 
}

# Entrenamos el modelo XGBoost
modelXGBoostPen = xgb.train(paramsPen, dtrainPen)

In [11]:
#Se calcula la lista de predicciones y de explicaciones del modelo XGBoost con el csv de Pen:
clon_model_XGboost_pen = copy.deepcopy(modelXGBoostPen)
prediccionesPenXG = clon_model_XGboost_pen.predict(dtestPen)
explicacionesPenXG = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_pen, rangosPen)
    explicacionesPenXG.append(explicacion)
print(explicacionesPenXG)

[array([[ 0.09298435,  0.02755988,  0.0034961 ,  0.03425123,  0.01751129,
        -0.05291253, -0.07220502, -0.06546525,  0.02430045,  0.12776606,
        -0.0049922 ,  0.03516951, -0.06048639, -0.13102549,  0.05119763,
        -0.02714961]]), array([[ 0.41121687,  0.16702742, -0.2191471 , -0.41479197, -0.02110666,
        -0.07817839,  0.29138876, -0.45246037,  0.10091965,  0.14960706,
         0.12032299,  0.24152077,  0.12403106, -0.21571483, -0.051135  ,
        -0.15350027]]), array([[-0.13552846,  0.14698446,  0.85887331,  0.18572069, -0.50883928,
         0.01651621, -0.09250281,  0.30558985,  0.37409016, -0.05323516,
        -0.03139618, -0.27175803, -0.12929183,  0.28034086, -0.25753122,
        -0.68803258]]), array([[-0.27519502,  0.01700098, -0.22603537,  0.01831133,  0.03660086,
         0.21714518, -0.00051076, -0.18701108,  0.19366287, -0.15533746,
        -0.19220386,  0.1477169 , -0.3853772 ,  0.33199935,  0.18994526,
         0.26928802]]), array([[-0.06905935,  0.145

In [12]:
#Se calcula la lista de predicciones y de explicaciones del modelo RandomForest con el csv de Pen:
prediccionesPenRF = modelRandomForestPen.predict(atributos_pruebaPen)
explicacionesPenRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestPen, rangosPen)
    explicacionesPenRF.append(explicacion)
print(explicacionesPenRF)

[array([[ 4.62414314e-32, -4.08190074e-32, -1.00136270e-32,
        -3.54221404e-32, -2.65137602e-32, -1.47644098e-32,
         1.43025400e-32,  1.19134013e-32, -2.91207643e-32,
         7.39498669e-33,  6.46542712e-33,  6.10684079e-33,
         5.69176411e-32, -3.37570393e-32, -2.63510197e-33,
        -6.64045396e-33]]), array([[-0.06012937, -0.0362186 , -0.08948586, -0.07671176, -0.04453666,
         0.13854949, -0.02492876,  0.1186769 , -0.03937813, -0.0484692 ,
        -0.02707561, -0.06078901, -0.07801235,  0.04530967,  0.15895128,
         0.12424797]]), array([[-0.23252706, -0.49189552,  0.40519652,  0.04481567, -0.29082629,
         0.19741591,  0.0773432 ,  0.18268994,  0.09279831,  0.16083796,
        -0.25756964, -0.05722154,  0.35733468,  0.42385587, -0.19171342,
        -0.42053457]]), array([[-0.34144945, -0.03106035, -0.11773673,  0.01889414,  0.07198947,
         0.17701715, -0.08857435,  0.08245662, -0.00425711, -0.2059989 ,
        -0.04572232,  0.02719271, -0.1778003

***VALIDACIÓN***

In [13]:
evaluate_model(modelRandomForestPen, atributos_entrenamientoPen, objetivo_entrenamientoPen, atributos_pruebaPen, objetivo_pruebaPen)

Puntuación media de validación cruzada: 0.9832669322709163
Puntuación en el conjunto de prueba: 0.9886777193691872


In [14]:
#las siguientes funciones sirven para eliminar atributos irrelevantes. 
#Un atributo es irrelevante si el peso de la explicación es cercano a 0


#También se calculan las listas de explicaciones alteradas, es decir, las explicaciones una vez eliminados los atributos irrelevantes
explicaciones_alteradas_RF_Bicis=[]
explicaciones_alteradas_RF_Pen=[]
explicaciones_alteradas_XG_Bicis=[]
explicaciones_alteradas_XG_Pen=[]

def eliminar_irrelevantes_RF(pesos, k, muestra, modelo,rangos):
    array = np.array(pesos).flatten() 
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k] #Calculamos los indices de los k atributos más cercanos a 0 en sus pesos
    copia = muestra.copy()
    for i in range(len(muestra)):
        if i in indices:
            copia[i] = 0 #Sustituimos el valor del atributo elegido por un 0
    muestra_2d = copia.reshape(1, -1)
    prediccion_nueva = modelo.predict(muestra_2d) #Se hace una nueva predicción con la muestra alterada
    dataframe = pd.DataFrame(muestra_2d)
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelo, rangos) #Se calcula la explicación
    if rangos == rangosBicis:
        explicaciones_alteradas_RF_Bicis.append(explicacion)
    else:
        explicaciones_alteradas_RF_Pen.append(explicacion)
    return prediccion_nueva



def eliminar_irrelevantes_XG(pesos, k, muestra, modelo,rangos):
    clon_model = copy.deepcopy(modelo)
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    
    #Extraemos los datos del DMatrix
    data = muestra.get_data()
    data_copy = data.copy()
    label = muestra.get_label()
    weight = muestra.get_weight()
    base_margin = muestra.get_base_margin()
    feature_names = muestra.feature_names

    
    for i in indices:
        data_copy[0,i]=0 #Sustituimos el valor del atributo elegido por un 0
    #Creamos una nueva DMatrix con los valores alterados
    copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
    dataframe = pd.DataFrame([copia.get_data().data], columns=feature_names)
    prediccion_nueva = clon_model.predict(copia)
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model, rangos)
    if rangos == rangosBicis:
        explicaciones_alteradas_XG_Bicis.append(explicacion)
    else:
        explicaciones_alteradas_XG_Pen.append(explicacion)
    return prediccion_nueva




In [15]:
predicciones_RF_Bicis = prediccionesBicisRF[:256]
objetivos_bicis = objetivo_pruebaBicis[:256]
explicaciones_RF_Bicis = explicacionesBicisRF

predicciones_XG_Bicis = prediccionesBicisXG[:256]
explicaciones_XG_Bicis = explicacionesBicisXG

predicciones_RF_Pen = prediccionesPenRF[:256]
objetivos_pen = objetivo_pruebaPen[:256]
explicaciones_RF_Pen = explicacionesPenRF

predicciones_XG_Pen = prediccionesPenXG[:256]
explicaciones_XG_Pen = explicacionesPenXG

In [16]:
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestPen = xgb.DMatrix(atributos_pruebaPen)

#En este bucle, se calculan las listas de predicciones de las muestras alteradas, es decir, con los atributos irrelevantes a 0
predicciones_alteradas_RF_Bicis=[]
predicciones_alteradas_RF_Pen=[]
predicciones_alteradas_XG_Bicis=[]
predicciones_alteradas_XG_Pen=[]

for i in range(256):
    
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Bicis[i],5,atributos_pruebaBicis[i],modelRandomForestBicis,rangosBicis)
    predicciones_alteradas_RF_Bicis.append(pred)
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Pen[i],5,atributos_pruebaPen[i],modelRandomForestPen,rangosPen)
    predicciones_alteradas_RF_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Pen[i],5,dtestPen.slice([i]),modelXGBoostPen,rangosPen)
    predicciones_alteradas_XG_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Bicis[i],5,dtestBicis.slice([i]),modelXGBoostBicis,rangosBicis)
    predicciones_alteradas_XG_Bicis.append(pred)
    
print(len(predicciones_alteradas_XG_Bicis))
print(len(predicciones_alteradas_XG_Pen))

256
256


***MÉTRICAS***

In [17]:
# Identidad

def identidad(i, muestra, lista_explicaciones):
    e = lista_explicaciones[i]
    return np.linalg.norm(muestra - muestra) != 0 or np.linalg.norm(e - e) == 0

# Separabilidad

def separabilidad(i,j,muestra_i,muestra_j, lista_explicaciones):
    e1 = lista_explicaciones[i]
    e2 = lista_explicaciones[j]
    return np.linalg.norm(muestra_i - muestra_j) == 0 or np.linalg.norm(e1 - e2) > 0 

# Estabilidad

def estabilidad(i, muestra, lista_muestras, lista_explicaciones):
    e = lista_explicaciones[i]
    d_muestras = np.array([np.linalg.norm(muestra - x) for x in lista_muestras])
    d_explicaciones = np.array([np.linalg.norm(e - x) for x in lista_explicaciones])
    return spearmanr(d_muestras, d_explicaciones)[0] > 0

# Selectividad

def selectividad(muestras, n, modelo, lista_objetivos, tipo):
    lista_auc = []
    for i in range(n):
        # segun el tipo de tarea el valor AUC se calculará de una forma u otra
        if tipo == "regresion":
            muestras_actualizadas = np.copy(muestras)
            # se asignan a 0 las i muestras
            muestras_actualizadas[:, i] = 0
            predicciones = modelo.predict(muestras_actualizadas)
            # AUC se calcula a través del error cuadrático medio
            auc = mean_squared_error(predicciones[:256], lista_objetivos)
            lista_auc.append(auc)
        elif tipo == "clasificacion":
            muestras_actualizadas = np.copy(muestras)
            # se asignan a 0 las i muestras
            muestras_actualizadas[:, i] = 0
            predicciones = modelo.predict_proba(muestras_actualizadas)
            auc = roc_auc_score(lista_objetivos, predicciones[:256], multi_class='ovr')
            lista_auc.append(auc)
        elif tipo == "xgboost":
            # datos de la matriz
            data = muestras.get_data()
            data_copy = data.copy()
            label = muestras.get_label()
            weight = muestras.get_weight()
            base_margin = muestras.get_base_margin()
            # se asignan a 0 las i muestras
            data_copy[0,i]=0
            # con los datos de la matriz creo una copia y extraigo sus predicciones
            copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
            predicciones = modelo.predict(copia)
            # AUC se calcula a través del error cuadrático medio
            auc = mean_squared_error(predicciones[:256], lista_objetivos)
            lista_auc.append(auc)
    # el valor final de AUC es la media de todos los AUC obtenidos
    return np.mean(lista_auc)

# Coherencia
def coherencia(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return np.abs(error_prediccion_original - error_prediccion_nuevo)

# Completitud

def completitud(lista_predicciones, lista_objetivo,explicaciones,explicaciones_actualizadas):
    error_prediccion = np.mean(lista_predicciones - lista_objetivo)
    error_explicacion = np.mean([explicaciones[i] - explicaciones_actualizadas[i] for i in range(256)])
    return error_explicacion / error_prediccion

# Congruencia
def congruencia(lista_predicciones, lista_objetivo,predicciones_actualizadas):
    sumatorio_coherencia = 0
    sumatorio_desviacion = 0
    # cálculo de la coherencia promedio
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_coherencia += alpha
    coherencia_promedio = sumatorio_coherencia / len(lista_predicciones)
    # cálculo del radicando de la raíz cuadrada
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_desviacion += np.power(alpha-coherencia_promedio,2)
    arg = sumatorio_desviacion / len(lista_predicciones)
    return np.sqrt(arg)

In [18]:
# Bicis Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaBicis[i],explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,atributos_pruebaBicis[i],atributos_pruebaBicis[255],explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaBicis[i],atributos_pruebaBicis[:256],explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

print("\nSelectividad:", selectividad(atributos_pruebaBicis,5,modelRandomForestBicis,objetivos_bicis,"regresion"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = completitud(predicciones_RF_Bicis,objetivos_bicis,explicaciones_RF_Bicis,explicaciones_alteradas_RF_Bicis)
print("\nCompletitud:", res_completitud)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Bicis, objetivos_bicis,predicciones_alteradas_RF_Bicis)[0])


Identidad: True

Separabilidad: True

Estabilidad: False

Selectividad: 5.364020703124999

Coherencia: 20.879453124999984

Completitud: -2.1943405389108384e-16

Congruencia: 68.22482433799978


In [19]:
# Bicis XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestBicis.slice([i]).get_data().data,explicaciones_XG_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,dtestBicis.slice([i]).get_data().data,dtestBicis.slice([255]).get_data().data,explicaciones_XG_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Selectividad

print("\nSelectividad:", selectividad(dtestBicis,5,modelXGBoostBicis,objetivos_bicis,"xgboost"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)[0]
print("\nCoherencia:", res_coherencia/256)

# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Bicis, objetivos_bicis,predicciones_alteradas_XG_Bicis)[0])


Identidad: True

Separabilidad: True

Selectividad: 90.91356044611857

Coherencia: 16.625524408183992

Completitud: -0.17850933186127804

Congruencia: 51.235012


In [20]:
# Pen Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaPen[i],explicaciones_RF_Pen)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,atributos_pruebaPen[i],atributos_pruebaPen[255],explicaciones_RF_Pen)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaPen[i],atributos_pruebaPen[:256],explicaciones_RF_Pen)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

print("\nSelectividad:", selectividad(atributos_pruebaPen,5,modelRandomForestPen,objetivos_pen,"clasificacion"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Pen, objetivos_pen,predicciones_alteradas_RF_Pen)[0])


Identidad: True

Separabilidad: True

Estabilidad: False

Selectividad: 0.9972501765678213

Coherencia: 1.19921875

Completitud: 0.0008521112354734902

Congruencia: 2.060453382061443


In [21]:
# Pen XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestPen.slice([i]).get_data().data,explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,dtestPen.slice([i]).get_data().data,dtestPen.slice([255]).get_data().data,explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Selectividad

print("\nSelectividad:", selectividad(dtestPen,5,modelXGBoostPen,objetivos_pen,"xgboost"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCompletitud", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Pen, objetivos_pen,predicciones_alteradas_XG_Pen)[0])


Identidad: True

Separabilidad: True

Selectividad: 0.5078125

Coherencia: 1.17578125

Completitud 0.001886485107434055

Congruencia: 2.252678
